In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bokeh.plotting
import catboost
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import train_test_split, StratifiedKFold
import sklearn.preprocessing, sklearn.feature_selection, sklearn.model_selection
from sklearn.pipeline import Pipeline, make_pipeline
import sklearn.base
from sklearn.base import BaseEstimator, TransformerMixin
import seaborn as sns
import dateutil.parser
import collections
import sklearn.utils
import itertools
import re

import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)

SEED = 42
np.random.seed = SEED

%matplotlib inline
%load_ext Cython

<IPython.core.display.Javascript object>

In [2]:
%%cython
cimport numpy as np

cpdef float qtof_(x):
    qtys = {
        'enough': 1.0,
        'insufficient': .6,
        'seasonal': .4,
        'dry': .2,
        'unknown': 0,
    }
    if x in qtys:
        return qtys[x]
    return 0

In [3]:
class LabelencodeAll:
    def __init__(self, cols):
        self.cols = cols
        self.encs_ = collections.defaultdict(sklearn.preprocessing.LabelEncoder)
    def fit(self, X, y=None):
        for c in self.cols:
            if c not in X.columns: continue
            if not np.issubdtype(X[c].dtype, np.number):
                self.encs_[c].fit(X[c])
        return self
    def transform(self, X):
        X = X.copy()
        for c, enc in self.encs_.items():
            if c not in X.columns: continue
            if not c in X.columns: continue
            if not np.issubdtype(X.loc[:,c].dtype, np.number):
                X.loc[:,c] = enc.transform(X.loc[:,c])
        return X

class DecodeQuantity(BaseEstimator, TransformerMixin):
    def __init__(self, col):
        self.col = col
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        X.loc[:,self.col] = X.loc[:,self.col].map(qtof_)
        return X
    
    
class HideMissingValues(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        for c in X.columns:
            if np.issubdtype(X.loc[:,c].dtype, np.number):
                med = X.loc[:,c].median()
                if np.isnan(med):
                    med = 0
                X.loc[:,c].fillna(med, inplace=True)
            elif set(X.loc[:,c].unique()).issubset({True, False, np.nan}):
                X.loc[X.loc[:,c] == 1, c] = 1
                X.loc[X.loc[:,c] != 1, c] = 0
                X.loc[:,c].fillna(0.5, inplace=True)
            else:
                X.loc[:,c].fillna('unknown', inplace=True)
                X.loc[:,c] = X.loc[:,c].astype('str')
        return X
    

class DropGarbage(BaseEstimator, TransformerMixin):
    def __init__(self, cols=['id', 'funder', 'recorded_by', 'date_recorded']):
        self.cols = cols
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.drop(self.cols, axis=1, errors='ignore')

class SplitDate(BaseEstimator, TransformerMixin):
    def __init__(self, col, drop=True):
        self.col = col
        self.drop = drop
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        def t(x):
            if isinstance(x, str):
                try:
                    x = dateutil.parser.parse(x)
                except:
                    return (np.nan, np.nan, np.nan, np.nan)
            elif pd.isnull(x):
                return (np.nan, np.nan, np.nan, np.nan)
            return (x.year, x.month, x.day, x.weekday())
        X = X.copy()
        (X.loc[:, self.col + '_year'],
         X.loc[:, self.col + '_month'],
         X.loc[:, self.col + '_day'],
         X.loc[:, self.col + '_weekday']) = zip(
        *X.loc[:, self.col].map(t))
        if self.drop:
            X.drop(self.col, axis=1, inplace=True)
        return X

    
class HandyFeatures(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        X.loc[:,'amount_per_man'] = X.loc[:,'amount_tsh']/X.loc[:,'population']
        return X

class OutcomeFrequences(BaseException, TransformerMixin):
    def __init__(self, groupby, drop=False, return_only_fq=False):
        self.groupby = groupby
        self.drop = drop
        self.cnts_ = None
        self.fq_cols_ = None
        self.return_only_fq = return_only_fq
        self.unknowns_ = 0
        
    def fit(self, X, y=None):
        possible_outcomes = [_.split()[-1] if isinstance(_, str) else _ for _ in np.unique(y)]
        n_outs = len(possible_outcomes)
        self.cnts_ = collections.defaultdict(lambda: np.zeros(n_outs))
        # print(self.groupby)
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        for i in range(X.shape[0]):
            # assuming $y$ is label-encoded
            keys, out = tuple(X.iloc[i, igroupby].values), y[i]
            self.cnts_[keys][out] += 1 # no of `(keys, out)` occurences
        for k in self.cnts_:
            self.cnts_[k] //= self.cnts_[k].sum()
            
        self.possible_outcomes_ = possible_outcomes
        self.fq_cols_ = ['_'.join(self.groupby + [str(out), 'fq'],) for out in possible_outcomes]
        return self
    
    def transform(self, X):
        self.unknowns_ = 0
        igroupby = np.array([X.columns.get_loc(_) for _ in self.groupby])
        new_cols = [tuple([None for out in self.possible_outcomes_]) for _ in range(X.shape[0])]
        for i in range(X.shape[0]):
            keys = tuple(X.iloc[i, igroupby].values)
            if keys in self.cnts_:
                new_cols[i] = tuple([self.cnts_[keys][out] for out in self.possible_outcomes_])
            else:
                self.unknowns_ += 1
        self.unknowns_ /= X.size
        # print(str(X)[:100])
        # print(str(new_cols)[:100])
        # print(str(self.fq_cols_[:100]))
        new_cols = pd.DataFrame(new_cols, columns=self.fq_cols_)
        for c in self.fq_cols_:
            new_cols.loc[:,c].fillna(0, inplace=True) # new_cols.loc[:,c].median(), inplace=True)
        if self.return_only_fq:
            return new_cols
        X = pd.concat((X, new_cols,), axis=1)
        if self.drop:
            X.drop(self.groupby, axis=1, inplace=True, errors='ignore')
        return X

class ExplicitAge(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X = X.copy()
        X.loc[:, 'age'] = X.loc[:,'date_recorded_year'] - X.loc[:,'construction_year']
        return X

def show_intersections(A, B, add_cat_feats=['date_recorded', 'construction_year',
                                            'date_recorded_year', 'date_recorded_month']):
    cat_features = np.where(np.array([(X_train[c].dtype == object)
                                  or (c.endswith('code'))
                                  or (c in add_cat_feats) for c in X_train.columns.values]))[0]
    cat_feats_observed = []
    for c in cat_features:
        n_uni = len(set(A.iloc[:,c]).union(B.iloc[:,c]))
        n_int = len(set(A.iloc[:,c]).intersection(B.iloc[:,c]))
        n_te = len(set(A.iloc[:,c]))
        cat_feats_observed.append((A.columns[c], n_int/n_uni, n_int/n_te))
    cat_feats_observed = (
        pd.DataFrame(cat_feats_observed, columns=['var', 'intersection/union', 'intersection/B'])
        .sort_values('intersection/B')
    )
    return cat_feats_observed

        
def split_train_cv_test(X, y, proportions=(.75, .25/2, .25/2)):
    # expecting $y$ to be numpy array
    outs = np.unique(y)
    proportions = np.array(proportions)
    classes = [np.where(y == i)[0] for i in outs]
    xparts = [[] for _ in proportions]
    yparts = [[] for _ in proportions]
    for cidx in classes:
        cidx = sklearn.utils.shuffle(cidx)
        cprops = cidx.size * proportions
        cprops = cprops.astype(int)
        cprops[-1] = cidx.size - cprops[:-1].sum()
        # print(cidx)
        cx = X.iloc[cidx,:]
        cy = y[cidx]
        for xpart, ypart, sz in zip(xparts, yparts, cprops):
            xpart.append(cx.iloc[:sz,:])
            ypart.append(cy[:sz])
            cx, cy = cx.iloc[sz:,:], cy[sz:]
    xparts = [pd.concat(xpart) for xpart in xparts]
    yparts = [np.concatenate(ypart) for ypart in yparts]
    parts = xparts + yparts
    return parts
def cat_features_indices(X):
    return np.where(np.array([(X[c].dtype == object)
        or (c.endswith('code'))                                                  
    for c in X.columns.values]))[0]

def cat_features_names(X):
    return X.columns[cat_features_indices(X)]

def copy_and_reset(X, cols, save=False, cat_unknown='unknown', num_unknown=0):
    X = X.copy()
    for c in X.columns:
        if save and c in cols:
            continue
        if not save and c not in cols:
            continue
        if X[c].dtype == object:
            X.loc[:,c] = cat_unknown
        else:
            X.loc[:,c] = num_unknown
    return X



In [4]:
def spawn_noisy(X, y, colsets_to_reset=[], colsets_to_save=None):
    X = X.drop('id', axis=1, errors='ignore').copy()
    colsets = colsets_to_reset if colsets_to_save is None else colsets_to_save
    X = pd.concat([copy_and_reset(X, cols,
                                  save=colsets_to_save is not None).reset_index(drop=True)
                  for cols in colsets],
                 ignore_index=True)
    y = np.concatenate([y
                  for cols in colsets])
    return X.reset_index(drop=True), y

In [5]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv')
X_test = pd.read_csv('X_test.csv')
display(show_intersections(X_train, X_test))
display(X_train.shape, y_train.shape, X_test.shape)

,var,intersection/union,intersection/B
3,wpt_name,0.055950,0.068342
5,subvillage,0.294315,0.326939
2,installer,0.343011,0.385368
1,funder,0.344699,0.388830
14,scheme_name,0.563960,0.599926
0,date_recorded,0.861789,0.893258
13,scheme_management,0.923077,0.923077
10,ward,0.930887,0.933556
17,extraction_type,0.944444,0.944444
7,region_code,0.962963,0.962963


(59400, 40)

(59400, 2)

(14850, 40)

In [6]:
X_train.loc[(X_train.loc[:,'source_type'] != X_train.loc[:,'source']) |
            (X_train.loc[:,'source_type'] != X_train.loc[:,'source_class']),
            ['source', 'source_type', 'source_class']].drop_duplicates()

,source,source_type,source_class
0,spring,spring,groundwater
1,rainwater harvesting,rainwater harvesting,surface
2,dam,dam,surface
3,machine dbh,borehole,groundwater
5,other,other,unknown
7,shallow well,shallow well,groundwater
16,river,river/lake,surface
66,hand dtw,borehole,groundwater
198,lake,river/lake,surface
3353,unknown,other,unknown


In [7]:
class PrefitedPipeline:
    def __init__(self, steps, verbose=1):
        self.steps = steps
        self.verbose = verbose
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        if self.verbose > 2:
            print('Input columns: %s' % X.columns)
        for stepname, step in self.steps:
            if self.verbose > 0:
                print('Executing %s' % stepname)
            X = step.transform(X)
            if self.verbose > 2:
                print('Step %s output columns: %s' % (stepname,
                                                      X.columns if isinstance(X, pd.DataFrame)
                                                      else type(X)))
        return X

In [8]:
y_enc = sklearn.preprocessing.LabelEncoder()
ytr = y_enc.fit_transform(y_train.iloc[:,1])

preprqty = DecodeQuantity('quantity')
xtr = preprqty.fit_transform(X_train)
xte = preprqty.fit_transform(X_test)

xtr.drop(['id', 'num_private', 'recorded_by', 'wpt_name'], axis=1, errors='ignore', inplace=True)
xte.drop(['id', 'num_private', 'recorded_by', 'wpt_name'], axis=1, errors='ignore', inplace=True)

preprdate = SplitDate('date_recorded', drop=True)
xtr = preprdate.fit_transform(xtr)
xte = preprdate.transform(xte)
preprage = ExplicitAge()
xtr = preprage.fit_transform(xtr)
xte = preprage.transform(xte)

xtr.loc[:,cat_features_names(xtr)] = 'unknown'
xte.loc[:,cat_features_names(xte)] = 'unknown'

xlabelenc = LabelencodeAll(cols=cat_features_names(xtr))
xlabelenc.fit(pd.concat((xtr, xte)))
xtr = xlabelenc.transform(xtr)
xte = xlabelenc.transform(xte)

hmv_ = HideMissingValues()
xtr = hmv_.fit_transform(xtr)
xte = hmv_.transform(xte)

hf_ = HandyFeatures()
xtr = hf_.fit_transform(xtr)
xte = hf_.transform(xte)

xtr.drop(['date_recorded_day', 'date_recorded_weekday'], axis=1, errors='ignore', inplace=True)
xte.drop(['date_recorded_day', 'date_recorded_weekday'], axis=1, errors='ignore', inplace=True)

fqpreprs = [(catcol + '_fq', OutcomeFrequences([catcol], drop=False))
    for catcol in ['installer', 'subvillage', 'basin', 'extraction_type_class',
                   'date_recorded_month', 'region', 'ward', 'source_type']
]
for i in range(len(fqpreprs)):
    print('fitting %s' % fqpreprs[i][0])
    xtr = fqpreprs[i][1].fit_transform(xtr, ytr)
    print('applying %s to test' % fqpreprs[i][0])
    xte = fqpreprs[i][1].transform(xte)


xtr, xcv, xva, ytr, ycv, yva = split_train_cv_test(xtr, ytr, proportions=(.7, .15, .15))


cols_to_reset = [
    [],
    ['date_recorded_month'],
    ['subvillage'],
    [ 'installer', 'funder'],
    [ 'subvillage', 'installer', 'funder', 'date_recorded_month' ],
    [ 'subvillage', 'installer', 'funder',
     'scheme_name', 'ward', 'region_code', ],
    [ 'subvillage', 'installer', 'funder',
     'scheme_management', 'ward', 'region_code', 'date_recorded_month'],
]

cols_to_save = [
    ['age', 'amount_per_man','extraction_type',
     'water_quality', 'quality_group', 'source', 'source_type', 'source_class',
     'waterpoint_type', 'waterpoint_type_group'],
    ['date_recorded_month', 'age', 'amount_tsh', 'population',
     'amount_per_man','extraction_type', 'longtitude', 'latitude'
     'water_quality', 'quality_group', 'source', 'source_type', 'source_class',
     'waterpoint_type', 'waterpoint_type_group'],
    ['date_recorded_month', 'age', 'amount_tsh', 'population',
     'amount_per_man','extraction_type', 'longtitude', 'latitude'
     'water_quality', 'quality_group', 'source', 'source_type', 'source_class',
     'waterpoint_type', 'waterpoint_type_group'],
    ['date_recorded_month', 'age', 'amount_tsh', 'population',
     'amount_per_man','extraction_type', 'longtitude', 'latitude'
     'water_quality', 'quality_group', 'source', 'source_type', 'source_class',
     'waterpoint_type', 'waterpoint_type_group'],
    ['date_recorded_month', 'age', 'amount_tsh', 'population',
     'amount_per_man','extraction_type', 'longtitude', 'latitude'
     'water_quality', 'quality_group', 'source', 'source_type', 'source_class',
     'waterpoint_type', 'waterpoint_type_group'],
    ['date_recorded_month', 'age', 'amount_tsh', 'population',
     'amount_per_man','extraction_type', 'longtitude', 'latitude'
     'water_quality', 'quality_group', 'source', 'source_type', 'source_class',
     'waterpoint_type', 'waterpoint_type_group']
]

fitting installer_fq
applying installer_fq to test
fitting subvillage_fq
applying subvillage_fq to test
fitting basin_fq
applying basin_fq to test
fitting extraction_type_class_fq
applying extraction_type_class_fq to test
fitting date_recorded_month_fq
applying date_recorded_month_fq to test
fitting region_fq
applying region_fq to test
fitting ward_fq
applying ward_fq to test
fitting source_type_fq
applying source_type_fq to test


In [9]:
xtr, ytr = [pd.concat((a.reset_index(drop=True), b.reset_index(drop=True),),
                      ignore_index=True) if isinstance(a, pd.DataFrame)
            else np.concatenate((a,b)) for a,b in zip(
    spawn_noisy(xtr, ytr, cols_to_reset),
    spawn_noisy(xtr, ytr, colsets_to_save=cols_to_save))]
                     
display('sizes: %s %s' % (xtr.index, ytr.size))

'sizes: RangeIndex(start=0, stop=540514, step=1) 540514'

In [10]:
xcv, ycv = spawn_noisy(xcv, ycv, colsets_to_save=[xcv.columns, xcv.columns] + cols_to_save)

In [11]:
xva, yva = spawn_noisy(xva, yva, colsets_to_save=[xva.columns, xva.columns] + cols_to_save)

In [12]:
cbc_cat_feats = [xtr.columns.get_loc(c) for c in [
    'funder', 'installer', 'basin', 'subvillage',
    'region', 'region_code', 'district_code', 'lga', 'ward',
    'extraction_type',
    'management', 'management_group', 'payment', 'payment_type', 'water_quality',
    'quality_group', 'source', 'source_type', 'source_class',
    'waterpoint_type', 'waterpoint_type_group'
]]

In [14]:
clf = catboost.CatBoostClassifier(iterations=2000,
                                  loss_function='MultiClass',
                                  eval_metric='Accuracy',
                                  calc_feature_importances=True)
clf.fit(xtr, ytr, eval_set=(xcv, ycv),
        cat_features=cbc_cat_feats,
        verbose=True)

Borders generated
feature 10 is redundant categorical feature, skipping it
feature 11 is redundant categorical feature, skipping it
feature 30 is redundant categorical feature, skipping it
feature 22 is redundant categorical feature, skipping it
feature 1 is redundant categorical feature, skipping it
feature 31 is redundant categorical feature, skipping it
feature 12 is redundant categorical feature, skipping it
feature 23 is redundant categorical feature, skipping it
feature 32 is redundant categorical feature, skipping it
feature 24 is redundant categorical feature, skipping it
feature 25 is redundant categorical feature, skipping it
feature 33 is redundant categorical feature, skipping it
feature 26 is redundant categorical feature, skipping it
feature 34 is redundant categorical feature, skipping it
feature 3 is redundant categorical feature, skipping it
feature 27 is redundant categorical feature, skipping it
feature 19 is redundant categorical feature, skipping it
feature 6 is re

63:	learn 0.6459721672	test 0.6260664571	bestTest 0.6261787158	passed: 1.18 sec	total: 1m 20s	remaining: 40m 21s
64:	learn 0.6460739222	test 0.6261506511	bestTest 0.6261787158	passed: 1.14 sec	total: 1m 21s	remaining: 40m 16s
65:	learn 0.646171977	test 0.6262909744	bestTest 0.6262909744	passed: 1.1 sec	total: 1m 22s	remaining: 40m 10s
66:	learn 0.646314434	test 0.6261506511	bestTest 0.6262909744	passed: 1.11 sec	total: 1m 23s	remaining: 40m 5s
67:	learn 0.6462774322	test 0.6261225864	bestTest 0.6262909744	passed: 1.14 sec	total: 1m 24s	remaining: 40m 1s
68:	learn 0.6463088838	test 0.6261225864	bestTest 0.6262909744	passed: 1.14 sec	total: 1m 25s	remaining: 39m 57s
69:	learn 0.6462885328	test 0.6263190391	bestTest 0.6263190391	passed: 1.09 sec	total: 1m 26s	remaining: 39m 51s
70:	learn 0.6462404304	test 0.6263751684	bestTest 0.6263751684	passed: 1.18 sec	total: 1m 27s	remaining: 39m 49s
71:	learn 0.6462681818	test 0.6263190391	bestTest 0.6263751684	passed: 1.34 sec	total: 1m 29s	remaini

136:	learn 0.650020166	test 0.6293921194	bestTest 0.6293921194	passed: 1.25 sec	total: 2m 45s	remaining: 37m 35s
137:	learn 0.6500571678	test 0.6294201841	bestTest 0.6294201841	passed: 1.13 sec	total: 2m 46s	remaining: 37m 33s
138:	learn 0.6500830691	test 0.6293921194	bestTest 0.6294201841	passed: 1.13 sec	total: 2m 48s	remaining: 37m 30s
139:	learn 0.6505714931	test 0.6297008307	bestTest 0.6297008307	passed: 1.15 sec	total: 2m 49s	remaining: 37m 28s
140:	learn 0.6504456869	test 0.6297008307	bestTest 0.6297008307	passed: 1.14 sec	total: 2m 50s	remaining: 37m 26s
141:	learn 0.6506343961	test 0.6297850247	bestTest 0.6297850247	passed: 1.14 sec	total: 2m 51s	remaining: 37m 24s
142:	learn 0.650773153	test 0.6297850247	bestTest 0.6297850247	passed: 1.15 sec	total: 2m 52s	remaining: 37m 22s
143:	learn 0.6504604876	test 0.6294622811	bestTest 0.6297850247	passed: 1.18 sec	total: 2m 53s	remaining: 37m 21s
144:	learn 0.6504253359	test 0.6295464751	bestTest 0.6297850247	passed: 1.1 sec	total: 2m 

209:	learn 0.6555038352	test 0.6337140772	bestTest 0.6337140772	passed: 0.915 sec	total: 4m 3s	remaining: 34m 38s
210:	learn 0.6556999449	test 0.6338544005	bestTest 0.6338544005	passed: 0.955 sec	total: 4m 4s	remaining: 34m 35s
211:	learn 0.6559034549	test 0.6338263359	bestTest 0.6338544005	passed: 0.928 sec	total: 4m 5s	remaining: 34m 32s
212:	learn 0.6560311111	test 0.6339947238	bestTest 0.6339947238	passed: 0.886 sec	total: 4m 6s	remaining: 34m 28s
213:	learn 0.656151367	test 0.6340227885	bestTest 0.6340227885	passed: 0.91 sec	total: 4m 7s	remaining: 34m 25s
214:	learn 0.6561550672	test 0.6339385945	bestTest 0.6340227885	passed: 0.922 sec	total: 4m 8s	remaining: 34m 22s
215:	learn 0.6563252756	test 0.6340227885	bestTest 0.6340227885	passed: 0.925 sec	total: 4m 9s	remaining: 34m 19s
216:	learn 0.656497334	test 0.6340789178	bestTest 0.6340789178	passed: 0.924 sec	total: 4m 10s	remaining: 34m 16s
217:	learn 0.6565454364	test 0.6340227885	bestTest 0.6340789178	passed: 0.915 sec	total: 4

282:	learn 0.6604787295	test 0.6371099012	bestTest 0.6371099012	passed: 0.972 sec	total: 5m 11s	remaining: 31m 31s
283:	learn 0.6605120311	test 0.6373203862	bestTest 0.6373203862	passed: 0.927 sec	total: 5m 12s	remaining: 31m 29s
284:	learn 0.6605231317	test 0.6373484508	bestTest 0.6373484508	passed: 0.922 sec	total: 5m 13s	remaining: 31m 27s
285:	learn 0.6606766892	test 0.6373624832	bestTest 0.6373624832	passed: 0.924 sec	total: 5m 14s	remaining: 31m 25s
286:	learn 0.6607617934	test 0.6374326448	bestTest 0.6374326448	passed: 0.908 sec	total: 5m 15s	remaining: 31m 22s
287:	learn 0.6607488428	test 0.6375308711	bestTest 0.6375308711	passed: 0.892 sec	total: 5m 16s	remaining: 31m 20s
288:	learn 0.6607062907	test 0.6377834531	bestTest 0.6377834531	passed: 0.93 sec	total: 5m 17s	remaining: 31m 18s
289:	learn 0.6607562431	test 0.6377413561	bestTest 0.6377834531	passed: 0.93 sec	total: 5m 18s	remaining: 31m 16s
290:	learn 0.6607284918	test 0.6377413561	bestTest 0.6377834531	passed: 0.991 sec	

...

1645:	learn 0.693726712	test 0.6516193309	bestTest 0.6519420745	passed: 0.917 sec	total: 27m 3s	remaining: 5m 49s
1646:	learn 0.6936823098	test 0.6515912663	bestTest 0.6519420745	passed: 0.917 sec	total: 27m 4s	remaining: 5m 48s
1647:	learn 0.6937692641	test 0.6518859452	bestTest 0.6519420745	passed: 0.911 sec	total: 27m 5s	remaining: 5m 47s
1648:	learn 0.6937748143	test 0.6518017512	bestTest 0.6519420745	passed: 0.921 sec	total: 27m 6s	remaining: 5m 46s
1649:	learn 0.6937711142	test 0.6518017512	bestTest 0.6519420745	passed: 0.877 sec	total: 27m 7s	remaining: 5m 45s
1650:	learn 0.6937692641	test 0.6518017512	bestTest 0.6519420745	passed: 0.9 sec	total: 27m 7s	remaining: 5m 44s
1651:	learn 0.6937748143	test 0.6518859452	bestTest 0.6519420745	passed: 0.867 sec	total: 27m 8s	remaining: 5m 43s
1652:	learn 0.6937988655	test 0.6518578806	bestTest 0.6519420745	passed: 0.897 sec	total: 27m 9s	remaining: 5m 42s
1653:	learn 0.6937933153	test 0.6519140099	bestTest 0.6519420745	passed: 0.889 sec	

1716:	learn 0.6943927447	test 0.6520403009	bestTest 0.6520403009	passed: 0.89 sec	total: 28m 7s	remaining: 4m 38s
1717:	learn 0.6943834942	test 0.6520403009	bestTest 0.6520403009	passed: 0.915 sec	total: 28m 8s	remaining: 4m 37s
1718:	learn 0.6944075454	test 0.6520122362	bestTest 0.6520403009	passed: 0.895 sec	total: 28m 9s	remaining: 4m 36s
1719:	learn 0.6943760939	test 0.6520122362	bestTest 0.6520403009	passed: 0.914 sec	total: 28m 10s	remaining: 4m 35s
1720:	learn 0.694479699	test 0.6520964302	bestTest 0.6520964302	passed: 0.907 sec	total: 28m 11s	remaining: 4m 34s
1721:	learn 0.6944667483	test 0.6520122362	bestTest 0.6520964302	passed: 0.917 sec	total: 28m 12s	remaining: 4m 33s
1722:	learn 0.694459348	test 0.6520683655	bestTest 0.6520964302	passed: 0.969 sec	total: 28m 13s	remaining: 4m 32s
1723:	learn 0.6944704485	test 0.6521385272	bestTest 0.6521385272	passed: 0.939 sec	total: 28m 14s	remaining: 4m 31s
1724:	learn 0.6944889494	test 0.6521385272	bestTest 0.6521385272	passed: 0.924

1787:	learn 0.6952271357	test 0.6524893354	bestTest 0.6525174001	passed: 0.892 sec	total: 29m 11s	remaining: 3m 27s
1788:	learn 0.6952104848	test 0.6525594971	bestTest 0.6525594971	passed: 0.909 sec	total: 29m 12s	remaining: 3m 26s
1789:	learn 0.6952326859	test 0.6525314324	bestTest 0.6525594971	passed: 0.906 sec	total: 29m 13s	remaining: 3m 25s
1790:	learn 0.6951864337	test 0.6524753031	bestTest 0.6525594971	passed: 0.888 sec	total: 29m 14s	remaining: 3m 24s
1791:	learn 0.6952437865	test 0.6524051414	bestTest 0.6525594971	passed: 0.913 sec	total: 29m 15s	remaining: 3m 23s
1792:	learn 0.6952400863	test 0.6523770768	bestTest 0.6525594971	passed: 0.916 sec	total: 29m 16s	remaining: 3m 22s
1793:	learn 0.6953325908	test 0.6524051414	bestTest 0.6525594971	passed: 0.889 sec	total: 29m 17s	remaining: 3m 21s
1794:	learn 0.6953510917	test 0.6523770768	bestTest 0.6525594971	passed: 0.947 sec	total: 29m 18s	remaining: 3m 20s
1795:	learn 0.6953584921	test 0.6524472384	bestTest 0.6525594971	passed:

1859:	learn 0.6959579215	test 0.652769982	bestTest 0.6529524023	passed: 0.892 sec	total: 30m 17s	remaining: 2m 16s
1860:	learn 0.6959856729	test 0.652769982	bestTest 0.6529524023	passed: 0.904 sec	total: 30m 17s	remaining: 2m 15s
1861:	learn 0.6959912232	test 0.652769982	bestTest 0.6529524023	passed: 0.923 sec	total: 30m 18s	remaining: 2m 14s
1862:	learn 0.6959764224	test 0.6527980467	bestTest 0.6529524023	passed: 0.898 sec	total: 30m 19s	remaining: 2m 13s
1863:	learn 0.6960134243	test 0.6527980467	bestTest 0.6529524023	passed: 0.879 sec	total: 30m 20s	remaining: 2m 12s
1864:	learn 0.6960152743	test 0.6527980467	bestTest 0.6529524023	passed: 0.9 sec	total: 30m 21s	remaining: 2m 11s
1865:	learn 0.6960134243	test 0.652769982	bestTest 0.6529524023	passed: 0.891 sec	total: 30m 22s	remaining: 2m 10s
1866:	learn 0.6960411756	test 0.6528401437	bestTest 0.6529524023	passed: 0.951 sec	total: 30m 23s	remaining: 2m 9s
1867:	learn 0.6961059288	test 0.6529243377	bestTest 0.6529524023	passed: 0.917 

1931:	learn 0.696884817	test 0.6531628873	bestTest 0.653190952	passed: 0.886 sec	total: 31m 22s	remaining: 1m 6s
1932:	learn 0.6969218189	test 0.6531628873	bestTest 0.653190952	passed: 0.967 sec	total: 31m 23s	remaining: 1m 5s
1933:	learn 0.6968977677	test 0.6531628873	bestTest 0.653190952	passed: 0.901 sec	total: 31m 23s	remaining: 1m 4s
1934:	learn 0.6969088682	test 0.6531628873	bestTest 0.653190952	passed: 0.897 sec	total: 31m 24s	remaining: 1m 3s
1935:	learn 0.6968533655	test 0.6530927256	bestTest 0.653190952	passed: 0.917 sec	total: 31m 25s	remaining: 1m 2s
1936:	learn 0.6968163637	test 0.6531207903	bestTest 0.653190952	passed: 0.903 sec	total: 31m 26s	remaining: 1m 1s
1937:	learn 0.6968570657	test 0.653064661	bestTest 0.653190952	passed: 0.908 sec	total: 31m 27s	remaining: 1m
1938:	learn 0.696862616	test 0.6529944993	bestTest 0.653190952	passed: 0.912 sec	total: 31m 28s	remaining: 59.4s
1939:	learn 0.6968126635	test 0.6530365963	bestTest 0.653190952	passed: 0.917 sec	total: 31m 2

In [15]:
display(clf.score(xtr, ytr),
        clf.score(xcv, ycv),
        clf.score(xva, yva))

0.69754529947420418

0.65348563089357881

0.65213428315010091

In [ ]:
orig_len = 59400 # X_train.shape[0]
display(clf.score(xcv.iloc[:orig_len,:], ycv[:orig_len]),)

In [ ]:
y_te = y_enc.inverse_transform(clf.predict(xte).astype(int))

In [ ]:
y_te

In [ ]:
ans = pd.DataFrame({'status_group': y_te.ravel()}, index=X_test['id'])
ans.index.name = 'id'
ans.to_csv('ans.csv')

In [ ]:
!head ans.csv

In [ ]:
# def encode_ans()

In [ ]:
for dsname, ds in [('xtr', xtr),
                   ('ytr', ytr),
                   ('')]